In [ ]:
import os
import numpy as np
import skimage.io as io
from balancer import get_class_counts_frequencies, balance
from matplotlib import pyplot as plt

In [ ]:
def plot_class_hist(y_masks, num_defects, save_path = None):
    if len(y_masks) == num_defects:
        counts = y_masks
    else:
        counts = get_class_counts_frequencies(y_masks, num_defects, with_ejection_class= False)[0]
    plt.ylabel("Number of Pixels")
    plt.xlabel("Defect Class")
    plt.xticks(rotation = 90)
    x_axis = range(1, num_defects+1)
    bars = plt.bar(x_axis, counts)
    plt.bar_label(bars)
    block = True
    if save_path:
        block = False
    plt.show(block=block)

    if save_path is not None:
        plt.savefig(save_path, bbox_inches = "tight")
        plt.close()
    

In [ ]:
orig_mask_data_dir = "/home/icon/shared_folders/streit/Jonas/2024-03-06_All_Samples_as_PNGs/train/masks"
orig_mask_paths = [os.path.join(orig_mask_data_dir, fn) for fn in  os.listdir(orig_mask_data_dir) if fn.endswith(".png")]
balanced_file_indices = balance(orig_mask_paths, 10000, 15, 128, 128, based_on_generated_data_only=True)

In [ ]:
values, counts = np.unique(balanced_file_indices, return_counts=True)

In [ ]:
len(values)

In [ ]:
np.max(counts)

In [ ]:
np.min(counts)

In [ ]:
plot_class_hist(orig_masks[balanced_file_indices], 14)

In [ ]:
np.unique(orig_masks, return_counts=True)[1]

In [ ]:
mask_dir = "RESULTS/20240315-194356_sample_2024-03-06_All_Samples_as_PNGs_d=300_c=15_samples=10000_model=ema_0.9999_053460.pt_s=1.5/good/labels"
mask_paths = [os.path.join(mask_dir, fn) for fn in  os.listdir(mask_dir)]
masks = np.zeros((len(mask_paths), 128, 128), np.uint8)

for i,p in enumerate(mask_paths):
    mask = (io.imread(p, as_gray=True) * 255).astype(np.uint8)
    masks[i] = mask

In [ ]:
plot_class_hist(masks, 14)

In [ ]:
plot_class_hist(masks, 14)

In [ ]:
np.unique(masks, return_counts=True)

In [ ]:
plot_class_hist([1947398,    402390,    459769,    833356,   1825384,
           721963,    264016,    864246,    131367,    344145,    563285,
          1857278,     20254,    271240], 14)

In [ ]:

import cv2
import numpy as np

def is_blueish(image_path):
    # Read the image
    image = cv2.imread(image_path)

    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define lower and upper thresholds for darkness and blueness
    # dark_lower = np.array([0, 50, 50], dtype=np.uint8)  # Adjusted for dark brownish or dark violet
    # dark_upper = np.array([30, 255, 50], dtype=np.uint8)  # Adjusted for dark brownish or dark violet
    blue_lower = np.array([100, 50, 50], dtype=np.uint8)
    blue_upper = np.array([140, 255, 255], dtype=np.uint8)

    # Create masks to filter out dark and blueish regions
    # dark_mask = cv2.inRange(hsv_image, dark_lower, dark_upper)
    blue_mask = cv2.inRange(hsv_image, blue_lower, blue_upper)

    # Check if there are dark or blueish pixels in the image
    # dark_pixels = cv2.countNonZero(dark_mask)
    blue_pixels = cv2.countNonZero(blue_mask)

    num_image_pixels = image.shape[0] * image.shape[1]
    # Determine if the image is very dark or very blueish
    # if dark_pixels > (num_image_pixels//2) or blue_pixels > (num_image_pixels // 2):
    if blue_pixels > (num_image_pixels // 2):
        return True
    else:
        return False


def is_dark(image_path):
    # Read the image
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Calculate histogram
    hist = cv2.calcHist([image], [0], None, [256], [0, 256])

    # Calculate total number of pixels
    total_pixels = image.shape[0] * image.shape[1]

    # Calculate cumulative histogram
    cumulative_hist = np.cumsum(hist)

    # Calculate cumulative percentage
    cumulative_percentage = cumulative_hist / total_pixels

    is_dark = np.argmax(cumulative_percentage > 0.8) < 65 #80 percent of pixels have value below 75
    
    return is_dark


def is_overexposed(image_path, threshold=240):
    """
    Determines if an image is overexposed by checking if a significant portion of the image is saturated.
    
    Args:
    - image_path (str): Path to the image file.
    - threshold (int): Threshold for pixel intensity (0-255). Pixels with intensity above this threshold are considered saturated.
    
    Returns:
    - bool: True if the image is overexposed, False otherwise.
    """
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Count the number of pixels above the threshold
    num_saturated_pixels = cv2.countNonZero(cv2.inRange(gray_image, threshold, 255))

    # Calculate the ratio of saturated pixels to total pixels
    total_pixels = gray_image.size
    saturation_ratio = num_saturated_pixels / total_pixels

    # If the ratio is above a certain threshold, consider the image overexposed
    overexposed_threshold = 0.05  # You can adjust this threshold as needed
    return saturation_ratio > overexposed_threshold


def check_image(image_path, brightness_threshold = 240, undersaturation_threshold = 50):

    status = "ok"

    ################################################################
    # Check overexposition
    ################################################################
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Count the number of pixels above the threshold
    num_saturated_pixels = cv2.countNonZero(cv2.inRange(gray_image, brightness_threshold, 255))

    # Calculate the ratio of saturated pixels to total pixels
    total_pixels = gray_image.size
    saturation_ratio = num_saturated_pixels / total_pixels

    # If the ratio is above a certain threshold, consider the image overexposed
    overexposed_threshold = 0.05  # You can adjust this threshold as needed

    is_overexposed = saturation_ratio > overexposed_threshold

    if is_overexposed:
        status = "overexposed"
    
    ################################################################
    # End Check overexposition
    ################################################################

    
    ################################################################
    # Check blueish
    ################################################################
    
    # Convert image to HSV color space
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    blue_lower = np.array([100, 50, 50], dtype=np.uint8)
    blue_upper = np.array([140, 255, 255], dtype=np.uint8)

    # Create masks to filter out dark and blueish regions
    blue_mask = cv2.inRange(hsv_image, blue_lower, blue_upper)

    # Check if there are dark or blueish pixels in the image
    blue_pixels = cv2.countNonZero(blue_mask)

    num_image_pixels = image.shape[0] * image.shape[1]
    
    is_blueish = blue_pixels > (num_image_pixels // 2)
    if is_blueish:
        if status == "ok":
            status = "blueish"
        else:
            status = status + "_blueish"
    
    ################################################################
    # End Check blueish
    ################################################################


    ################################################################
    # Check darkness
    ################################################################

    # Calculate histogram
    hist = cv2.calcHist([gray_image], [0], None, [256], [0, 256])

    # Calculate total number of pixels
    total_pixels = gray_image.shape[0] * gray_image.shape[1]

    # Calculate cumulative histogram
    cumulative_hist = np.cumsum(hist)

    # Calculate cumulative percentage
    cumulative_percentage = cumulative_hist / total_pixels

    is_dark = np.argmax(cumulative_percentage > 0.8) < 65 #80 percent of pixels have value below 75

    if is_dark:
        if status == "ok":
            status = "dark"
        else:
            status = status + "_dark"
    
    ################################################################
    # End Check darkness
    ################################################################
            
    ################################################################
    # Check undersaturation
    ################################################################
    
     # Calculate mean saturation level
    mean_saturation = np.mean(hsv_image[:,:,1])
    
    # Determine if image is undersaturated
    is_undersaturated = mean_saturation < undersaturation_threshold
    if is_undersaturated:
        if status == "ok":
            status = "undersaturated"
        else:
            status = status + "_undersaturated"
    
    ################################################################
    # End Check undersaturation
    ################################################################
    
    
    return status


In [ ]:
# Read the image
image_path = "RESULTS/20240313-173943_sample_2024-03-06_All_Samples_as_PNGs_d=300_c=15_samples=10000_model=ema_0.9999_053460.pt_s=1.5/samples/0035_373.png"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Calculate histogram
hist = cv2.calcHist([image], [0], None, [256], [0, 256])

# Calculate total number of pixels
total_pixels = image.shape[0] * image.shape[1]

# Calculate cumulative histogram
cumulative_hist = np.cumsum(hist)

# Calculate cumulative percentage
cumulative_percentage = cumulative_hist / total_pixels

# Find first and last bins with cumulative percentage above a threshold (adjust threshold as needed)
start_bin = np.argmax(cumulative_percentage > 0.01)
end_bin = np.argmax(cumulative_percentage > 0.99)

# Calculate contrast
contrast = end_bin - start_bin

In [ ]:
np.argmax(cumulative_percentage > 0.8)

In [ ]:
contrast

In [ ]:
start_bin

In [ ]:
def get_out_string(img_path):
    out_str = ""
    
    if is_overexposed(img_path):
        out_str += ", overexposed"
    
    if is_blueish(img_path):
        out_str += ", blue"
    
    if is_dark(img_path):
        out_str += ", dark"
    
    if out_str == "":
        out_str = "good"

    return out_str
    
    
folder_path = "RESULTS/20240313-173943_sample_2024-03-06_All_Samples_as_PNGs_d=300_c=15_samples=10000_model=ema_0.9999_053460.pt_s=1.5/samples"
img_path = os.path.join(folder_path, "0014_0.png")
print(get_out_string(img_path)) #good

img_path = os.path.join(folder_path, "0006_0.png")
print(get_out_string(img_path)) #dark

img_path = os.path.join(folder_path, "0007_0.png")
print(get_out_string(img_path)) #dark

img_path = os.path.join(folder_path, "0008_371.png") #dark
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0009_0.png") #blue
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0026_1.png") #overexposed
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0033_373.png") #blue
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0043_2.png") #dark and blue
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0046_2.png") #blue
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0065_375.png") #dark
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0036_2.png") #good
print(get_out_string(img_path))

img_path = os.path.join(folder_path, "0035_373.png") #good
print(get_out_string(img_path))


In [ ]:
import os
folder_path = "RESULTS/20240315-194356_sample_2024-03-06_All_Samples_as_PNGs_d=300_c=15_samples=10000_model=ema_0.9999_053460.pt_s=1.5/samples"
image_paths = [p for p in os.listdir(folder_path) if p.endswith(".png")]
image_states = [check_image(os.path.join(folder_path,path)) for path in image_paths]
np.unique(image_states, return_counts=True)

In [ ]:
# folder_path = "RESULTS/20240315-194356_sample_2024-03-06_All_Samples_as_PNGs_d=300_c=15_samples=10000_model=ema_0.9999_053460.pt_s=1.5/labels"

In [ ]:
import shutil
for path, state in zip(image_paths, image_states):
    full_path = os.path.join(folder_path, path)
    if not os.path.isdir(os.path.join(folder_path, os.pardir, state, os.path.basename(folder_path))):
        os.makedirs(os.path.join(folder_path, os.pardir, state, os.path.basename(folder_path)))
    shutil.copyfile(full_path, os.path.join(folder_path, os.pardir, state, os.path.basename(folder_path), path))

In [ ]:
orig_masks = np.zeros((len(orig_mask_paths), 128, 128), np.uint8)
for i,p in enumerate(orig_mask_paths):
    mask = (io.imread(p, as_gray=True)).astype(np.uint8)
    orig_masks[i] = mask

In [ ]:
np.unique(orig_masks)

In [ ]:
plot_class_hist(orig_masks, 14)

In [ ]:
all_masks = np.append(orig_masks, masks, axis=0)
plot_class_hist(all_masks, 14)

In [ ]:
plot_class_hist(all_masks[:(len(orig_masks)+10000)], 14)

In [ ]:
#tmp

In [ ]:
classes, indices, counts = np.unique(masks[np.nonzero(masks)], return_index=True, return_counts=True)

In [ ]:
classes.dtype

In [ ]:
indices

In [ ]:
counts.dtype

In [ ]:
 Sampled 3454 masks. Class counts are [1701935  394098  427655  774944 1669687  614158  274631 1421449  193695
  277820  595360 1857278   18015  261344]

In [ ]:
np.unique(masks[:3454], return_counts=True)

In [ ]:
np.unique(orig_masks, return_counts=True)

In [ ]:
1006805 + 741956

In [ ]:
get_class_counts_frequencies(masks, 14)[0]

In [ ]:
np.unique(masks[np.nonzero(masks)], return_counts=True)[1]

In [ ]:
balanced_file_indices[-2]

In [ ]:
orig_mask_paths[2262]

In [ ]:
files, counts = np.unique(file_list, return_counts=True)

In [ ]:
np.any(counts > 1)

In [ ]:
indices, idx_counts = np.unique(balanced_file_indices[-3631:], return_counts=True)

In [ ]:
np.any(idx_counts > 1)

In [ ]:
len(file_list)

In [ ]:
file_list[1]

In [ ]:
file_list[2]

In [ ]:
import platform
cpu_model = platform.processor()
print("CPU model:", cpu_model)

In [ ]:
#end tmp